<a href="https://colab.research.google.com/github/fazeelibtesam/Scraper/blob/main/Web_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install requests beautifulsoup4 nltk

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from collections import Counter

brand_urls = [
    "http://districtbarnyc.com/",
    "http://www.patrizias.com/",
    "https://www.eatitaliannyc.com/",
    "http://saporiristorante.com/",
    "http://www.joycestavern.com/"
]

def get_about_us(url):
    try:
        if not url.startswith("http"):
            url = "https://" + url

        #load the home page
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')

        #link that likely leads to "About Us"
        about_url = None
        for link in soup.find_all('a', href=True):
            if "about" in link['href'].lower():
                about_url = link['href']
                break

        #If not found, fallback to main page content
        if not about_url:
            paragraphs = soup.find_all('p')
            return " ".join(p.get_text() for p in paragraphs)[:1000]

        #Construct full About URL if relative
        if not about_url.startswith("http"):
            about_url = url.rstrip('/') + '/' + about_url.lstrip('/')

        about_response = requests.get(about_url)
        about_soup = BeautifulSoup(about_response.text, 'html.parser')
        paragraphs = about_soup.find_all('p')
        about_text = " ".join([p.get_text() for p in paragraphs])

        return about_text[:1000] if about_text else "No About Us content found."

    except Exception as e:
        return f"Error loading site: {e}"
basic_stopwords = set([
    'the', 'and', 'for', 'are', 'with', 'that', 'this', 'was', 'from', 'you',
    'your', 'have', 'has', 'our', 'but', 'not', 'all', 'can', 'about', 'will',
    'we', 'they', 'their', 'them', 'more', 'who', 'what', 'where', 'how', 'when'
])

def extract_keywords(text):
    words = re.findall(r'\b[a-z]{3,}\b', text.lower())
    words = [word for word in words if word not in basic_stopwords]
    common_words = Counter(words).most_common(10)
    return [word for word, _ in common_words]

def generate_intro(keywords):
    if len(keywords) < 3:
        return "Not enough data to generate intro."
    return f"Our brand focuses on {', '.join(keywords[:3])}, and specializes in {', '.join(keywords[3:6])} to deliver exceptional experiences."

for url in brand_urls:
    print(f"\n Scraping: {url}")

    about_text = get_about_us(url)
    print("\n About Us Content (Excerpt):")
    print(about_text)

    keywords = extract_keywords(about_text)
    print("\n Extracted Keywords:")
    print(keywords)

    content = generate_intro(keywords)
    print("\n Generated Brand Intro:")
    print(content)



 Scraping: http://districtbarnyc.com/

 About Us Content (Excerpt):
From the creators of The Hop Shoppe, Richmond Republic and The Pizza Parlor. District is your local neighborhood spot where Staten Islanders get their taste of beers, custom and classic cocktails, and a flavor packed menu. We take pride in the products we source and the dishes we serve. Take a look at our menu. Great food made with quality ingredients. 2 Sneden Ave, Staten Island, NY 10312(718) 984-3700

 Extracted Keywords:
['staten', 'menu', 'take', 'creators', 'hop', 'shoppe', 'richmond', 'republic', 'pizza', 'parlor']

 Generated Brand Intro:
Our brand focuses on staten, menu, take, and specializes in creators, hop, shoppe to deliver exceptional experiences.

 Scraping: http://www.patrizias.com/

 About Us Content (Excerpt):
Traditional family dining for 25 years Founders, chefs and brothers, Giacomo and Gennaro Alaio were both born in Naples, Italy and came to the United States when they were just 8 and 10 years 